<a href="https://colab.research.google.com/github/mak109/cs6910_assignment2/blob/main/PART%20B/cs6910_assignment2_partB_question1_2_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading and Fine-tuning pretrained Models

In [2]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=641dfac7d106c9052d6d1b9f00b851db8d9509e7f244cef03713d094daed63aa
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [3]:
!pip install wandb -qqq
import wandb
wandb.login()

     |████████████████████████████████| 1.7 MB 5.2 MB/s 
     |████████████████████████████████| 144 kB 58.1 MB/s 
     |████████████████████████████████| 181 kB 53.7 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


KeyboardInterrupt: ignored

In [24]:
from wandb.keras import WandbCallback

In [25]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import numpy as np
import random
import wget
import os
from zipfile import ZipFile
from PIL import Image
from matplotlib import gridspec
plt.rcParams["figure.figsize"] = (20,10)

In [26]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,Sequential,regularizers,optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import * 
import random
random.seed(123)

In [27]:
import wget
import os
from inspect import *
from zipfile import ZipFile
url='https://storage.googleapis.com/wandb_datasets/nature_12K.zip'
filename = os.path.basename(url)

if not os.path.exists(filename) and not os.path.exists("inaturalist_12K"):
  filename = wget.download(url)
  with ZipFile(filename, 'r') as zip:
    print('Extracting all the files now...')
    zip.extractall()
    print('Done!')
  os.remove(filename)
train_dir = 'inaturalist_12K/train'
test_dir = 'inaturalist_12K/val'

In [28]:
image_size = (256,256)
num_classes = 10

In [29]:
#Creating dictionary of models based on imagenet 
model_list = dict()
for key,value in getmembers(tf.keras.applications,isfunction):
    model_list[key] = value

In [55]:
def CNN(config):
    base_model = model_list[config["model"]](input_shape=image_size +(3,),include_top=False,weights="imagenet")
    base_model.trainable = True #this is important
    if(len(base_model.layers) > config['fine_tune_last']):
        for layer in base_model.layers[:-config['fine_tune_last']]:
            layer.trainable = False    
    global_average_layer = layers.GlobalAveragePooling2D()
    prediction_layer = layers.Dense(num_classes,activation='softmax')
    inputs = layers.Input((image_size[0],image_size[1],3))
    input_rescale=layers.Rescaling(1./255)(inputs)
    x = base_model(input_rescale)    
    x = global_average_layer(x)
    x = layers.Dropout(config['dropout'])(x)
    outputs = prediction_layer(x)
    model = keras.Model(inputs,outputs)
    return model

In [58]:
def train():


  #Default parameters

    config_ = {
    "model": 'InceptionV3',
    "learning_rate": 1e-4,
    "data_augment": "True",
    "dropout":0.2,
    "batch_size":32,
    "fine_tune_last":20,
    "epochs":3
    }

    wandb.init(config=config_)
    config = wandb.config
    
    #Setting run name for better readability
    wandb.run.name = "model_"+str(config["model"])+"bs_"+str(config["batch_size"])+"epoch_"+str(config["epochs"])

    val_generator = ImageDataGenerator(dtype=tf.float32,validation_split=0.1,data_format='channels_last').flow_from_directory(
        train_dir,
        target_size = image_size,
        batch_size = config['batch_size'],
        color_mode = 'rgb',
        class_mode = 'sparse',
        shuffle=True,
        subset='validation',
        seed=123
    
    )


    #Data Augmentation
    if config["data_augment"] == 'True':
        data_generator = ImageDataGenerator(
        rotation_range=50, #random rotation between -50(clockwise) to 50(anti-clockwise) degree
        brightness_range=(0.2,0.8), 
        zoom_range=0.3, #zoom in range from [0.7,1.3]
        horizontal_flip=True,
        vertical_flip=True,
        width_shift_range=0.1, #Horizontal Shifting as a ratio of width
        height_shift_range=0.2,#Vertical Shifting as a ratio of height
        data_format='channels_last',
        validation_split=0.1,
        dtype=tf.float32
        )
    else:
        data_generator = ImageDataGenerator(
            data_format='channels_last',
            validation_split=0.1,
            dtype=tf.float32
        )
    #Train set creation after conditional augmentation
    train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size = image_size,
    batch_size = config['batch_size'],
    color_mode = 'rgb',
    class_mode = 'sparse',
    shuffle=True,
    subset='training',
    seed=123
    )
    
    model = CNN(config)
    
    model.compile(
    optimizer=optimizers.Adam(learning_rate=config["learning_rate"]),
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
    )
    
    history = model.fit(train_generator,
                epochs=config['epochs'],
                validation_data=val_generator,
                callbacks = [WandbCallback()]
                )
    model.save(os.path.join(wandb.run.dir,"model.h5"))
    wandb.finish()
    return history,model

In [36]:
config = {
    "model": 'InceptionV3',
    "learning_rate": 1e-4,
    "data_augment": "False",
    "dropout":0.2,
    "batch_size":32,
    "fine_tune_last":20,
    "epochs":3
    }
model = CNN(config)

In [62]:
sweep_config = {
  "name" : "best-sweep-for-pretrained-model",
  "method" : "bayes",
  "metric" : {
      "name" : "val_accuracy",
      "goal" : "maximize"
  },
  
  "parameters" : {
      "model" : {
          "values" : ["InceptionV3", "InceptionResNetV2", "ResNet50", "Xception"]},

  "learning_rate" :{
      "values" : [1e-4]
  },
  "data_augment" : {
      "values" : ["False"]
  },
  "dropout" : {
      "values" : [0.2]
  },

  "batch_size" : {
      "values" : [32]
  },
  "fine_tune_last" : {
  "values" : [20]
  },
    "epochs" : {
      "values" : [5,10]
    }}
}

In [63]:
sweep_id=wandb.sweep(sweep_config,entity="dlstack",project="CS 6910 Assignment 2 Part B")

Create sweep with ID: nizh7n9i
Sweep URL: https://wandb.ai/dlstack/CS%206910%20Assignment%202%20Part%20B/sweeps/nizh7n9i


In [1]:
wandb.agent(sweep_id, function=train, count=4)

NameError: ignored

In [12]:
history,model = train()

Found 999 images belonging to 10 classes.
Found 9000 images belonging to 10 classes.
Epoch 1/3


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


282/282 [==============================] - 1353s 5s/step - loss: 0.9126 - accuracy: 0.7151 - val_loss: 0.6899 - val_accuracy: 0.7848
Epoch 2/3
282/282 [==============================] - 1356s 5s/step - loss: 0.5966 - accuracy: 0.8129 - val_loss: 0.6449 - val_accuracy: 0.7928
Epoch 3/3
282/282 [==============================] - 1378s 5s/step - loss: 0.4881 - accuracy: 0.8443 - val_loss: 0.6363 - val_accuracy: 0.7968


In [13]:
val_generator = ImageDataGenerator(dtype=tf.float32,validation_split=0.1,data_format='channels_last').flow_from_directory(
        train_dir,
        target_size = image_size,
        batch_size = config['batch_size'],
        color_mode = 'rgb',
        class_mode = 'sparse',
        shuffle=True,
        subset='validation',
        seed=123
    
    )
    #Data Augmentation
if config["data_augment"] == 'True':
    data_generator = ImageDataGenerator(
        rotation_range=50, #random rotation between -50(clockwise) to 50(anti-clockwise) degree
        brightness_range=(0.2,0.8), 
        zoom_range=0.3, #zoom in range from [0.7,1.3]
        horizontal_flip=True,
        vertical_flip=True,
        width_shift_range=0.1, #Horizontal Shifting as a ratio of width
        height_shift_range=0.2,#Vertical Shifting as a ratio of height
        data_format='channels_last',
        validation_split=0.1,
        dtype=tf.float32
        )
else:
    data_generator = ImageDataGenerator(
            data_format='channels_last',
            validation_split=0.1,
            dtype=tf.float32
        )
    #Train set creation after conditional augmentation
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size = image_size,
    batch_size = config['batch_size'],
    color_mode = 'rgb',
    class_mode = 'sparse',
    shuffle=True,
    subset='training',
    seed=123
    )
fine_tune_epochs = 5
total_epochs =  config['epochs'] + fine_tune_epochs

history_fine = model.fit(train_generator,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=val_generator)

Found 999 images belonging to 10 classes.
Found 9000 images belonging to 10 classes.
Epoch 3/8
 64/282 [=====>........................] - ETA: 16:24 - loss: 0.3685 - accuracy: 0.8854

KeyboardInterrupt: ignored

In [ ]:
test_generator = ImageDataGenerator(dtype=tf.float32,validation_split=0.0,data_format='channels_last').flow_from_directory(
        'inaturalist_12K/val',
        target_size = image_size,
        batch_size = 200,
        color_mode = 'rgb',
        class_mode = 'sparse',
        shuffle=False,
        seed=123
        )
loss0, accuracy0 = model.evaluate(test_generator)